In [1]:
import geopandas as gpd
import pandas as pd
slim = gpd.read_file("tokyo_slim.geojson")
pop = pd.read_csv("estat/tblT001101H5339.csv", skiprows=[1], encoding="shift_jis")

/var/folders/r2/8crk13rd1136ng21r3d8twg00000gn/T/ipykernel_95381/2325373055.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  pop = pd.read_csv("estat/tblT001101H5339.csv", skiprows=[1], encoding="shift_jis")


In [2]:
print("slim columns:", slim.columns.tolist())
print("pop columns:", pop.columns.tolist())

slim columns: ['KEY_CODE', '2023_building_count', '2023_office_count', '2023_office_total_use_area', '1996_building_count', '2001_building_count', '2006_building_count', '2011_building_count', '2016_building_count', '2023_old__building_count', '1996_office_count', '2001_office_count', '2006_office_count', '2011_office_count', '2016_office_count', '2023_old__office_count', '1996_office_total_use_area', '2001_office_total_use_area', '2006_office_total_use_area', '2011_office_total_use_area', '2016_office_total_use_area', '2023_old__office_total_use_area', '2023_building_count_percentile', '2023_office_count_percentile', '2023_office_total_use_area_percentile', '1996_building_count_percentile', '2001_building_count_percentile', '2006_building_count_percentile', '2011_building_count_percentile', '2016_building_count_percentile', '2023_old__building_count_percentile', '1996_office_count_percentile', '2001_office_count_percentile', '2006_office_count_percentile', '2011_office_count_percentil

In [3]:
# join pop into slim using KEY_CODE
slim['KEY_CODE'] = slim['KEY_CODE'].astype(int)
pop['KEY_CODE'] = pop['KEY_CODE'].astype(int)
slim_pop = slim.merge(pop, on="KEY_CODE", how="left")


In [4]:
# show how many rows succeeded and failed
print("Rows in slim:", len(slim))
print("Rows in pop:", len(pop))
print("Rows after merge:", len(slim_pop))

Rows in slim: 2489
Rows in pop: 16671
Rows after merge: 2489


In [5]:
# show how many rows have T001101001 data
print("Rows with T001101001:", slim_pop[slim_pop['T001101001'].notnull()].shape[0])

Rows with T001101001: 2317


In [6]:
# from slim_pop, eliminate columns that have "percentile" in their name
slim_pop = slim_pop.loc[:, ~slim_pop.columns.str.contains("percentile")]
# and rename T001101001 to "population"
slim_pop = slim_pop.rename(columns={"T001101001": "population"})
# rename T001101004 to pop_0_14
slim_pop = slim_pop.rename(columns={"T001101004": "pop_0_14"})
# rename T001101010 to pop_15_64
slim_pop = slim_pop.rename(columns={"T001101010": "pop_15_64"})
# rename T001101019 to pop_65_plus
slim_pop = slim_pop.rename(columns={"T001101019": "pop_65_plus"})
# rename T001101031 to pop_foreign
slim_pop = slim_pop.rename(columns={"T001101031": "pop_foreign"})
# rename T001101034 to households
slim_pop = slim_pop.rename(columns={"T001101034": "households"})
# rename T001101002 to pop_male
slim_pop = slim_pop.rename(columns={"T001101002": "pop_male"})
# rename T001101003 to pop_female
slim_pop = slim_pop.rename(columns={"T001101003": "pop_female"})

# delete all other T001101 columns
slim_pop = slim_pop.loc[:, ~slim_pop.columns.str.startswith("T001101")]

In [7]:
# delete columns 'HTKSYORI', 'HTKSAKI', 'GASSAN'
slim_pop = slim_pop.drop(columns=['HTKSYORI', 'HTKSAKI', 'GASSAN'], errors='ignore')

In [8]:
# ensure that the population columns are numeric
for col in ['population', 'pop_0_14', 'pop_15_64', 'pop_65_plus', 'pop_foreign', 'households']:
    slim_pop[col] = pd.to_numeric(slim_pop[col], errors='coerce')

In [9]:
slim_pop

,KEY_CODE,2023_building_count,2023_office_count,2023_office_total_use_area,1996_building_count,2001_building_count,2006_building_count,2011_building_count,2016_building_count,2023_old__building_count,...,2023_other_total_use_area,geometry,population,pop_male,pop_female,pop_0_14,pop_15_64,pop_65_plus,pop_foreign,households
0,533925552,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,...,NaN,"POLYGON ((139.7 35.54167, 139.69375 35.54167, ...",3301.0,1716.0,1585.0,670.0,2205.0,406.0,142.0,1398.0
1,533925554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"POLYGON ((139.7 35.54583, 139.69375 35.54583, ...",3665.0,1882.0,1783.0,500.0,2190.0,949.0,99.0,1613.0
2,533925561,664.0,16.0,13706.9320,529.0,473.0,488.0,492.0,584.0,547.0,...,2816.670,"POLYGON ((139.70625 35.54167, 139.7 35.54167, ...",4211.0,2061.0,2150.0,575.0,2456.0,1098.0,101.0,1862.0
3,533925562,1071.0,15.0,2347.4700,967.0,939.0,905.0,901.0,1008.0,880.0,...,3208.360,"POLYGON ((139.7125 35.54167, 139.70625 35.5416...",6385.0,3334.0,3051.0,627.0,3945.0,1596.0,162.0,3223.0
4,533925563,557.0,1.0,157.1200,485.0,485.0,479.0,486.0,529.0,466.0,...,112.950,"POLYGON ((139.70625 35.54583, 139.7 35.54583, ...",1731.0,890.0,841.0,246.0,1066.0,364.0,68.0,809.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2484,533957301,1190.0,22.0,9229.1405,934.0,1016.0,1088.0,1113.0,1145.0,1112.0,...,3337.123,"POLYGON ((139.88125 35.775, 139.875 35.775, 13...",3912.0,1993.0,1919.0,468.0,2509.0,910.0,80.0,1813.0
2485,533957303,269.0,4.0,2315.3280,214.0,241.0,254.0,252.0,258.0,248.0,...,210.720,"POLYGON ((139.88125 35.77917, 139.875 35.77917...",1622.0,807.0,815.0,236.0,946.0,428.0,37.0,625.0
2486,533957304,247.0,6.0,1704.0085,192.0,157.0,163.0,213.0,228.0,229.0,...,1170.750,"POLYGON ((139.8875 35.77917, 139.88125 35.7791...",1055.0,514.0,541.0,163.0,651.0,230.0,30.0,388.0
2487,533957401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"POLYGON ((139.88125 35.78333, 139.875 35.78333...",1798.0,930.0,868.0,259.0,1079.0,435.0,73.0,767.0


In [10]:
# export to geojson
slim_pop.to_file("tokyo_slim_pop.geojson", driver="GeoJSON")